In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV file
# Replace 'chienloup.csv' with the exact path to your file
data = pd.read_csv('chienloup.csv', sep=';')

# Extract quantitative and qualitative variables names
quantitative_vars = data.columns[1:-1]  # Quantitative variables
qualitative_var = data.columns[-1]     # Qualitative variable (e.g., category)

# Group the data by the qualitative variable
groups = data.groupby(qualitative_var)

# Initialize a dictionary to store intra-class covariance matrices
W_k_dict = {}

# Initilize the intraclass variance matrix
W = np.zeros((len(quantitative_vars), len(quantitative_vars)))

# Compute W_k for each group as well as W
for group_name, group in groups:
    group_values = group[quantitative_vars].values
    group_mean = np.mean(group_values, axis=0)
    n_k = group_values.shape[0]  # Number of samples in the group
    # Calculate W_k: intra-class covariance matrix
    W_k = (group_values - group_mean).T @ (group_values - group_mean) / n_k
    W_k_dict[group_name] = W_k  # Store each W_k with the corresponding group name
    W += W_k * n_k

W = W/data.shape[0]

###(Bonus)############################################################################################################################
# We compute and visualize the correlation matrix from the covariance matrix to interpret the values of the intra covariances
#for group_name, W_k in W_k_dict.items():
#    print(f"Intra-class correlation matrix W_k for group '{group_name}':")
#    # Computation of the correlation matrix from the covariance matrix
#    std_dev = np.sqrt(np.diag(W_k))  # Standard deviations
#    correlation_matrix = W_k / np.outer(std_dev, std_dev)  # Normalize to get correlations
#
#    # Visualization of the correlation matrix
#    plt.figure(figsize=(8, 6))
#    plt.imshow(correlation_matrix, cmap="coolwarm", vmin=-1, vmax=1)
#    plt.colorbar(label="Correlation")
#    plt.xticks(ticks=range(len(quantitative_vars)), labels=quantitative_vars, rotation=45, ha="right")
#    plt.yticks(ticks=range(len(quantitative_vars)), labels=quantitative_vars)
#    plt.title("Correlation Matrix Heatmap")
#    plt.tight_layout()
#    plt.show()
#
####################################################################################################################################



# Computation of inter-class covariance matrix (B), which measures variability between group means
overall_mean = np.mean(data[quantitative_vars].values, axis=0)  # Overall mean vector
B = np.zeros((len(quantitative_vars), len(quantitative_vars)))  # Initialize inter-class covariance matrix

for name, group in groups:
    group_size = len(group)  # Number of samples in the current group
    group_mean = np.mean(group[quantitative_vars].values, axis=0)  # Mean vector of the group
    diff = (group_mean - overall_mean).reshape(-1, 1)  # Difference vector reshaped as a column
    B += group_size * (diff @ diff.T)  # Weighted outer product of the difference vector

# Computation of total variance matrix (V) as the sum of intra-class (W) and inter-class (B) covariances
V = W + B

# Displaying intra-class (W), inter-class (B), and total variance (V) matrices
print(f"Total intra-class covariance matrix (W):")
print(pd.DataFrame(W, index=quantitative_vars, columns=quantitative_vars))
print("\n")
print("\nTotal inter-class covariance matrix (B):")
print(pd.DataFrame(B, index=quantitative_vars, columns=quantitative_vars))
print("\nTotal variance matrix (V):")
print(pd.DataFrame(V, index=quantitative_vars, columns=quantitative_vars))

Total intra-class covariance matrix (W):
              LCB           LSM            LBM         LP           LM    \
LCB    70166.944444  36126.587302  7473.134921  2907.956349  2022.908730   
LSM    36126.587302  20150.396825  1862.301587  1526.706349  1034.325397   
 LBM    7473.134921   1862.301587  7990.753968   482.718254   260.146825   
LP      2907.956349   1526.706349   482.718254   204.831349   108.974206   
LM      2022.908730   1034.325397   260.146825   108.974206    93.399603   
LAM     2907.531746   1442.341270   513.103175   149.091270   115.103968   

               LAM  
LCB    2907.531746  
LSM    1442.341270  
 LBM    513.103175  
LP      149.091270  
LM      115.103968  
LAM     361.976984  



Total inter-class covariance matrix (B):
               LCB            LSM             LBM          LP            LM    \
LCB    188171.666667  135466.666667  51011.666667  60854.166667  26479.500000   
LSM    135466.666667   97523.809524  36723.809524  43809.523810  19062.85

Coumputation of V^-1.B and its eigen values

In [17]:
# Compute the trace of B and V
trace_B = np.trace(B)  # Sum of diagonal elements of B
trace_V = np.trace(V)  # Sum of diagonal elements of V

# Display the results
print(f"Trace of Inter-class covariance matrix (B): {trace_B}")
print(f"Trace of Total variance matrix (V): {trace_V}")

# Compare the traces
print(f"Proportion of variance explained by inter-class variability: {trace_B / trace_V:.2%}")

Trace of Inter-class covariance matrix (B): 333191.90952380927
Trace of Total variance matrix (V): 432160.2126984125
Proportion of variance explained by inter-class variability: 77.10%


Interpretation of V, W, B:
The inter-class variance, which is the trace of B, accounts for 0.77% of the total variance. This shows that the original variables separate the classes' groups quite well.

Computation of V^-1 * B

In [18]:
# Compute V^-1 * B
V_inv = np.linalg.inv(V)
V_inv_B = V_inv @ B

# Compute the eigenvalues of V^-1 * B
eigenvalues, _ = np.linalg.eig(V_inv_B)

# Display the eigenvalues and their sum
print("Eigenvalues of V^-1 * B:\n", eigenvalues)
print("\nSum of eigenvalues:", np.sum(eigenvalues))

Eigenvalues of V^-1 * B:
 [ 9.94925042e-01+0.00000000e+00j  2.27611276e-14+0.00000000e+00j
  1.38879934e-14+0.00000000e+00j -7.47444178e-15+3.77105478e-15j
 -7.47444178e-15-3.77105478e-15j -1.41794016e-15+0.00000000e+00j]

Sum of eigenvalues: (0.9949250416627787+0j)


2) Etudiez les matrices V-1 et B : dimension, caractéristiques : Pouvez toujours diagonaliser directement 𝑉
−1𝐵 ?

In [19]:
# Study the matrices
print("Dimension of V^-1:", V_inv.shape)
print("Dimension of B:", B.shape)


# Study V^-1 * B
print("Dimension of V^-1 * B:", V_inv_B.shape)

# Compute the rank of V^-1 * B
rank_V_inv_B = np.linalg.matrix_rank(V_inv_B)

# Compute the condition number (high values indicate collinearity)
cond_number = np.linalg.cond(V_inv_B)

# Display results
print("Rank of V^-1 * B:", rank_V_inv_B)
print("Condition number of V^-1 * B:", cond_number)


Dimension of V^-1: (6, 6)
Dimension of B: (6, 6)
Dimension of V^-1 * B: (6, 6)
Rank of V^-1 * B: 4
Condition number of V^-1 * B: 9773931263870940.0


Commentary on V-1.B diagonalisation

In [20]:
# We observe that the rank computed for V^-1 * B is 4, while its size is 6x6. ()
# This indicates that the collinearity between its columns is so high that the computer estimates it as a set of nearly collinear vectors.
# This is further confirmed by the very high value of the condition number.

# Additionally, in the above computation of the eigenvalues of V^-1 * B, we observed that some eigenvalues were complex numbers.
# This indicates numerical instability, which suggests that the matrix is close to being non-invertible.

# In conclusion, V^-1 * B is not always invertible due to high collinearity between its columns.
# This is caused by the low rank of B, which determines the rank of V^-1 * B. 
# In this case, since the number of classes is q = 2, the rank of B is at most q-1 = 1.
# The above computed value of 4 for V^-1.B rank is due to wrong approximation made by the computer of the eigen values that are = 0 but estimated as slightly higher.

3) Si vous choisissez de diagonalisation cette matrice, utiliser le complément sur la diagonalisation donnée en cours
(technique de régularisation). Apres diagonalisation comme en ACP vous obtiendrez la projection des individus dans
un nouvel espace par une méthode factorielle par projections.

In [ ]:

# Step 1: Use SVD for decomposition of B into CC^T
U, S, _ = np.linalg.svd(B)  # SVD of B
C = U @ np.diag(np.sqrt(S))  # Construct C

# Compute the matrix C'Σ^(-1)C
C_T = C.T  # Transpose of C
C_Sigma_inv_C_T = C_T @ V_inv @ C

# Check if the matrix is symmetric (it should be)
# print("Is C'Σ^(-1)C symmetric?:", np.allclose(C_Sigma_inv_C_T, C_Sigma_inv_C_T.T))

# Diagonalize the symmetric matrix C'Σ^(-1)C
eigenvalues, eigenvectors = np.linalg.eigh(C_Sigma_inv_C_T)  # eigh ensures symmetric diagonalization

# Compute the transformation vector v using equation 
v_vectors = V_inv @ C @ eigenvectors

# Results
print("Eigenvalues of :", eigenvalues)
print("Eigenvectors (w) of C'Σ^(-1)C:")
print(eigenvectors)

print("Transformed eigenvectors (v):")
print(v_vectors)


Eigenvalues (λ): [1.39501568e-16 6.42928785e-16 8.80691405e-15 1.52139095e-14
 3.95012658e-14 9.94925042e-01]
Eigenvectors (w) of C'Σ^(-1)C:
[[ 3.11466899e-09 -1.01562193e-08 -6.31130972e-09 -7.35489693e-09
  -2.92605273e-10  1.00000000e+00]
 [-3.51895541e-02  2.94038595e-01  3.85270679e-01  3.96720101e-01
  -7.78769969e-01  8.21745009e-09]
 [-2.52545321e-01  6.34973955e-01 -8.71517872e-02  5.40267569e-01
   4.83264376e-01  1.08005059e-08]
 [-4.30715455e-01 -6.89746255e-02  8.21882155e-01 -2.37350143e-01
   2.79108099e-01  4.16415015e-09]
 [ 7.83736365e-01  4.41446214e-01  3.20679128e-01 -2.47354135e-01
   1.63900515e-01  2.29494447e-09]
 [-3.67728245e-01  5.57420059e-01 -2.56214150e-01 -6.58183072e-01
  -2.34964184e-01  2.79964834e-10]]
Transformed eigenvectors (v):
[[-2.30161932e-11  3.01017848e-11  7.72894203e-10  2.17918757e-10
  -3.08972750e-09  3.22180872e-04]
 [-3.18459924e-11  1.03842008e-10 -1.58585698e-09 -6.73051874e-10
   5.45242490e-09 -7.28432556e-06]
 [ 1.05137773e-10  2

Projection of X onto the space spanned by eigen vectors explaining at least 80% of the variance

In [ ]:
# Step 2: Calculate the explained variance
explained_variance_ratio = eigenvalues / np.sum(eigenvalues)  # Explained variance ratio
cumulative_explained_variance = np.cumsum(explained_variance_ratio)  # Cumulative explained variance

# Find the number of components needed to explain at least 80% of the variance
n_components = np.argmax(cumulative_explained_variance >= 0.8) + 1

print(f"Number of components to explain 80% of variance: {n_components}")
print("Cumulative explained variance:", cumulative_explained_variance)

# Step 3: Select the top n_components eigenvectors
selected_v_vectors = v_vectors[:, :n_components]

# Step 4: Project the data onto the selected eigenvectors
Z = X @ selected_v_vectors  # X is the centered data matrix

# Step 5: Visualize the projection
plt.scatter(Z[:, 0], Z[:, 1], c=y, cmap='viridis', edgecolor='k')  # y contains class labels
plt.title("Projection on top components explaining >= 80% variance")
plt.xlabel("Component 1")
plt.ylabel("Component 2")
plt.colorbar(label="Class")
plt.grid()
plt.show()